# OpenAI

Marvin includes first-class utilities for working with OpenAI's API. It's familiar, cuts boiler-plate, and - most importantly - optional. It's the type of stuff you'd build after your first or second refactor. Fingers crossed it saves you as much dev time as it's saved us. 

If you're looking for something higher-level, check out the rest of the docs. This section is for folks who prefer to work with a lower-level API but still appreciate a little syntactic sugar.

## ChatCompletion

Marvin includes a subclass of OpenAI's SDK implementation of ChatCompletion. This utility is completely standalone: you're free to use it with or without Marvin's other components or framework.

### Frozen Keyword Arguments

In `openai.ChatCompletion`, you normally have to pass keyword arguments to each invocation of `create` and `acreate`. With *Marvin*, you can choose to pass these keywords to the constructor of `marvin.openai.ChatCompletion`, and have those keywords passed to each subsequent invocation. This let's you do stuff like:

In [ ]:
import marvin

marvin.openai.ChatCompletion(model="gpt-3.5-turbo").create(
    messages=[{"role": "user", "content": "Tell me a joke!"}]
)

This small quality-of-life change let's you do some pretty great things, like:

#### Creating Model Facets

By freezing keyword arguments this lets us define, premission, and version `facets` of ChatGPT. This let's us define separate model instances for internal, public, or customer use. By combining this with function calling (see below), we can also give smart models access to smarter models when they intuit that it's prudent.

In [ ]:
import marvin

# Define a smart model.
gpt3 = marvin.openai.ChatCompletion(model="gpt-3.5-turbo")

# Define a smarter model.
gpt4 = marvin.openai.ChatCompletion(model="gpt-4")

# gp4.create(**kwargs) ~ openai.ChatCompletion.create(model = 'gpt-4', **kwargs)

Above, each instance passes its respective `model` keyword argument to create and acreate on each invocation. 

#### Facet Inheritence

Of course, these model configurations are chainable. 

In [ ]:
import marvin

pirate_system_message = {"role": "system", "content": "You talk like a pirate"}

# Define a smart, public model.
smart_pirate = gpt3(messages=[pirate_system_message]).create(
    [{"role": "user", "content": "Hello!"}]
)

By default, keyword arguments in create and acreate override frozen parameters. This default has two exceptions: messages and functions, wherein passed messages are concatenated with frozen messages (and likewise for functions).

## Functions

OpenAI's ChatCompletion API enables you to pass a list of `functions` for it to optionally call in service of a query. If it chooses to execute a function, either by choice or instruction, it will return the function's name along with its formatted parameters for *you* to evaluate. The OpenAI schema accepts a JSON Schema representation of your functions.

Marvin includes lightweight utilities for working with OpenAI's function API. These utilities are completely standalone: you're free to use them with or without Marvin's other components or framework. We expose `openai_fn`, a function decorator that makes working with OpenAI functions straightforward.

In [ ]:
from marvin.openai import openai_fn


@openai_fn
def add(x: int, y: int) -> str:
    return x + y

### Serialization

*Marvin* allows auto creation of JSON Schemas from functions:

In [ ]:
from marvin.openai import openai_fn


@openai_fn
def add(x: int, y: int) -> str:
    """Adds two numbers together"""
    return x + y


add.schema()

Returns its JSON Schema to use with OpenAI's function API.

In [ ]:
{
    "name": "add",
    "description": "Adds two numbers together",
    "parameters": {
        "x": {"type": "int", "description": null},
        "y": {"type": "int", "description": null},
        "required": ["x", "y"],
    },
}

### Evaluation

When ChatGPT decides it needs to invoke a function in service of your query, it will return the name of the function it would like you to invoke and the arguments to evaluate it with. *Marvin* provides means to evaluate a function from a ChatGPT Function Call Response. 

In [ ]:
import openai
from marvin.openai import openai_fn


@openai_fn
def add(x: int, y: int) -> str:
    """Adds two numbers together"""
    return x + y


response = openai.ChatCompletion.create(
    messages=[{"role": "user", "content": "What is 123123 + 85858?"}]
)

add.from_response(response) == 208981

### Registry

In most function-calling applications, you'll want to pass a list of several functions. The developer experience, accordingly, gets `n` times worse. We introduce a standard Function Registry to make things a little easier. It doubles as an API Router if you're into that sort of thing.

#### Serialization

In [ ]:
from marvin.openai import OpenAIFunctionRegistry

registry = OpenAIFunctionRegistry()


@registry.register
def add(x: int, y: int) -> int:
    """Adds two numbers together"""
    return x + y


@registry.register
def subtract(x: int, y: int) -> int:
    """Subtracts `y` from `x`"""
    return x - y


registry.schema()

Which yields a list of schemas which can be passed as a keyword argument to OpenAI's SDK.

In [ ]:
[
    {
        "name": "add",
        "description": "Adds two numbers together",
        "parameters": {
            "type": "object",
            "properties": {
                "x": {"title": "X", "type": "integer"},
                "y": {"title": "Y", "type": "integer"},
            },
            "required": ["x", "y"],
        },
    },
    {
        "name": "subtract",
        "description": "Subtracts `y` from `x`",
        "parameters": {
            "type": "object",
            "properties": {
                "x": {"title": "X", "type": "integer"},
                "y": {"title": "Y", "type": "integer"},
            },
            "required": ["x", "y"],
        },
    },
]

#### Evaluation

When ChatGPT decides it needs to invoke one of the functions in your registry in service of your query, it will return the name of the function it would like you to invoke and the arguments to evaluate it with. *Marvin* provides means to evaluate a function call in your registry dictated by a ChatGPT Function Call Response. 

In [ ]:
from marvin.openai import OpenAIFunctionRegistry

registry = OpenAIFunctionRegistry()


@registry.register
def add(x: int, y: int) -> int:
    """Adds two numbers together"""
    return x + y


@registry.register
def subtract(x: int, y: int) -> int:
    """Subtracts `y` from `x`"""
    return x - y


response = openai.ChatCompletion.create(
    messages=[{"role": "user", "content": "What is 123123 - 85858?"}]
)

registry.from_response(response) == 37265

#### Composability

Given two function routers, you can easily compose them. This lets you separately define, say, one registry devoted to accessing and processing one data source, and another devoted to accessing and processing another (with stricter permissions, perhaps). Including them is as straightfoward as calling `include` (which is, of course, idempotent).

In [ ]:
math = OpenAIFunctionRegistry()

arithmetic = OpenAIFunctionRegistry()

trigonometry = OpenAIFunctionRegistry()


@arithmetic.register
def add(x: int, y: int) -> int:
    """Adds two numbers together"""


@trigonometry.register
def tan(theta: float) -> float:
    """Calculates the tangent of `theta`."""
    return x - y


math.include(arithmetic)
math.include(trigonometry)

math.schema()

In [ ]:
{
    "functions": [
        {
            "name": "add",
            "description": "Adds two numbers together",
            "parameters": {
                "type": "object",
                "properties": {
                    "x": {"title": "X", "type": "integer"},
                    "y": {"title": "Y", "type": "integer"},
                },
                "required": ["x", "y"],
            },
        },
        {
            "name": "tan",
            "description": "Calculates the tangent of `theta`.",
            "parameters": {
                "type": "object",
                "properties": {"theta": {"title": "Theta", "type": "number"}},
                "required": ["theta"],
            },
        },
    ],
    "function_call": "auto",
}

### Code Generation

*Marvin* offers an experimental utility to author code using OpenAI's function API. Given a function with a typed signature and a docstring, we can write entire functions in the language of your choice. Simple call `.code()` on a function decorated with @openai_fn. 

Behind the scenes, we define a utility function write_code:

In [ ]:
from marvin.openai import openai_fn


@openai_fn
def write_code(
    language: str,
    filename: str,
    name: str,
    docstring: str,
    code: str,
) -> str:
    """Accepts and checks expertly staff engineer quality written `code` in `language`"""
    return (language, filename, name, docstring, code)

When you call the `code` method on *your* openai_fn, we simply call:

In [ ]:
@openai_fn
def add(x: int, y: int) -> str:
    """Adds two numbers together"""
    # There is no code here! #


{
    "messages": [
        {
            "role": "user",
            "content": f"""A function in python that described by the following schema:\n {add.schema}""",
        },
    ],
    **write_code.schema,
}

This returns the following payload for you to send to OpenAI, which forces it to write code that satisfies the high-level description of your function.

In [ ]:
{
    'messages': 
    [{
        'role': 'user',
        'content': "A function in python that described by the following schema:
            {'name': 'add', 
             'description': 'Adds two numbers together', 
             'parameters': {
                 'type': 'object', 
                 'properties': {
                     'x': {'title': 'X', 'type': 'integer'}, 
                     'y': {'title': 'Y', 'type': 'integer'}
                }, 'required': ['x', 'y']
            }
        }"
    }],
    'functions': [
        {
            'name': 'write_code',
            'description': 'Accepts and checks expertly staff engineer quality written `code` in `language`',
            'parameters': {
                'type': 'object',
                'properties': {
                    'language': {'title': 'Language', 'type': 'string'},
                    'filename': {'title': 'Filename', 'type': 'string'},
                    'name': {'title': 'Name', 'type': 'string'},
                    'docstring': {'title': 'Docstring', 'type': 'string'},
                    'code': {'title': 'Code', 'type': 'string'}},
            'required': ['language', 'filename', 'name', 'docstring', 'code']
            }
        }
    ],
    'function_call': {'name': 'write_code'}
}


If we give it a trivial example and call `.code()`, we see:

In [ ]:
from marvin.openai import openai_fn
import openai

openai.api_key = "YOUR_OPENAPI_KEY"


@openai_fn
def add(x: int, y: int) -> str:
    """Adds two numbers together"""
    # There is no code here! #


write_code_instructions = add.code()

response = await openai.ChatCompletion.acreate(
    model="gpt-3.5-turbo", **write_code_instructions
)

print(response.choices[0].message.get("function_call").get("arguments"))

Which returns

In [ ]:
{
    "language": "python",
    "filename": "add.py",
    "name": "add",
    "docstring": "Adds two numbers together",
    "code": "def add(x: int, y: int) -> int:\n    return x + y",
}